In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.statespace.varmax import VARMAX

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.tools import diff
%matplotlib inline
import statsmodels.api as sm
from statsmodels.tsa.stattools import acovf, acf, pacf, pacf_yw, pacf_ols
import warnings
warnings.filterwarnings('ignore')
from pandas.plotting import lag_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AR, ARResults, AutoReg
from statsmodels.graphics.tsaplots import month_plot, quarter_plot
from pmdarima import auto_arima
from statsmodels.tsa.arima_model import ARMA, ARMAResults, ARIMAResults
# from sm.tsa.statespace import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.eval_measures import rmse
import lightgbm as lgbm
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10)

np.random.seed(192)
from pylab import rcParams
import scipy.stats as stats
rcParams['figure.figsize'] = (12,5)
rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12
rcParams['axes.labelsize'] = 12

In [2]:
cols = ['STC_LANE', 'BKG_DT', 'TOTAL_ACTUAL_WT']
oct_18_dec_20 = pd.read_csv('.\data\Oct2018_Dec2020_BM_v6.csv', usecols = cols)
jan_21_jul_22 = pd.read_csv('.\data\Jan2021_Sept2022_BM_v6.csv', usecols = cols)
oct_18_dec_20.columns = oct_18_dec_20.columns.str.lower()
jan_21_jul_22.columns = jan_21_jul_22.columns.str.lower()
df = pd.concat([oct_18_dec_20, jan_21_jul_22])

In [3]:
cols_list = ['OU_CODE', 'STATE', 'ZONE_OU_CODE']
zone = pd.read_csv('.\data\STC_v2.csv', usecols = cols_list)
df['origin_stc'], df['destination_stc'] = df['stc_lane'].str.split('_', 1).str
stcs = set(list(zone['OU_CODE']))
origin_stcs = set(list(df['origin_stc'].unique()))
destination_stcs = set(list(df['destination_stc'].unique()))
origin_stc_list = zone[['OU_CODE', 'STATE', 'ZONE_OU_CODE']].copy()
destination_stc_list = zone[['OU_CODE', 'STATE', 'ZONE_OU_CODE']].copy()
origin_stc_list.rename(columns = {'OU_CODE': 'origin_stc', 'STATE': 'origin_state', 'ZONE_OU_CODE': 'origin_zone'},
                       inplace=True)
destination_stc_list.rename(columns = {'OU_CODE': 'destination_stc', 'STATE': 'dest_state', 'ZONE_OU_CODE': 'dest_zone'},
                            inplace=True)
dfz = pd.merge(df, origin_stc_list, on="origin_stc", how="left")
df1 = pd.merge(dfz, destination_stc_list, on="destination_stc", how="left")
# df1 = df1[df1['origin_stc']!=df1['destination_stc']]
df1.drop(columns=['origin_stc',
       'destination_stc', 'origin_state', 'origin_zone', 'dest_state',
       'dest_zone'], inplace=True)
df = df1.copy()
df['bkg_date'] = pd.to_datetime(df['bkg_dt'], format='%Y-%m-%d', errors='coerce')
df.drop(columns=['bkg_dt'], inplace=True)
df['weight'] = (df['total_actual_wt']/1000)
df.drop(columns=['total_actual_wt'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [4]:
df.head()

,stc_lane,bkg_date,weight
0,AMC_AMC,2018-10-01,22.90742
1,AMC_AMC,2018-10-02,4.59282
2,AMC_AMC,2018-10-03,32.32647
3,AMC_AMC,2018-10-04,28.89591
4,AMC_AMC,2018-10-05,23.47256


In [5]:

def wave(row):
    wave = ""
    if (row["bkg_date"] >= pd.Timestamp("2018-09-20")) and (row["bkg_date"] <= pd.Timestamp("2020-02-28")):
        wave="before covid"
    elif (row["bkg_date"] > pd.Timestamp("2020-02-28")) and (row["bkg_date"] <= pd.Timestamp("2020-06-15")):
        wave="1st wave"
    elif (row["bkg_date"] > pd.Timestamp("2020-06-15"))  and (row["bkg_date"] <= pd.Timestamp("2020-09-30")):
        wave="between waves"
    elif (row["bkg_date"] > pd.Timestamp("2020-09-30")) and (row["bkg_date"] <= pd.Timestamp("2021-01-15")):
        wave="2nd wave"
    elif (row["bkg_date"] > pd.Timestamp("2021-01-15")):
        wave="after second wave"
    return wave

def label_mth (row):
    if row['week_number'] == 1 :
        return 'begin_week'
    if row['week_number'] == 5:
        return 'end_week'
    return 'mid_week'

def label_yr (row):
    if row['bkg_date_month'] == 6 :
        return 'lean_mth'
    if row['bkg_date_month'] == 5:
        return 'lean_mth'
    if row['bkg_date_month'] == 4 :
        return 'lean_mth'
    if row['bkg_date_month'] == 1:
        return 'lean_mth'
    if row['bkg_date_month'] == 9 :
        return 'peak_mth'
    if row['bkg_date_month'] == 10:
        return 'peak_mth'
    if row['bkg_date_month'] == 11:
        return 'peak_mth'
    if row['bkg_date_month'] == 12:
        return 'peak_mth'
    return 'neu_mth'

In [6]:
def datetime_feats(df):
    for col in ['bkg_date']:
        df[col+'_week'] = df[col].dt.week
        df[col+'_month'] = df[col].dt.month
        df[col+'_quarter'] = df[col].dt.quarter
        df[col+'_year'] = df[col].dt.year
        df[col+'_day'] = df[col].dt.day
        df[col+'_dayofweek'] = df[col].dt.dayofweek
        df[col+'_dayname'] = df[col].dt.day_name()
        df[col+'_day_of_year'] = df[col].dt.dayofyear
        df[col+'_week_of_year'] = df[col].dt.weekofyear
        
    df['season'] = df.bkg_date_month%12 // 5 + 1
    df['bkg_date_semester'] = np.where(df['bkg_date_quarter'].isin([1,2]), 1, 2)
    df['bkg_date_is_weekend'] = np.where(df['bkg_date_dayname'].isin(['Sunday', 'Saturday']), 1, 0)    
    df['bkg_date_is_sunday'] = np.where(df['bkg_date_dayname'].isin(['Sunday']), 1, 0)
    df['bkg_date_is_wed'] = np.where(df['bkg_date_dayname'].isin(['Wednesday']), 1, 0)
    df['bkg_date_is_thu'] = np.where(df['bkg_date_dayname'].isin(['Thursday']), 1, 0)
    df['bkg_date_is_wed_thu'] = np.where(df['bkg_date_dayname'].isin(['Thursday', 'Wednesday']), 1, 0)
    df['week_number'] = (df['bkg_date_day'] - 1) // 7 + 1


#     df['month_name'] = df.apply(lambda row: label_month(row), axis=1)

    df["covid_wave"]=df.apply(lambda row:wave(row),axis=1)

    df['label_mth'] = df.apply(lambda row: label_mth(row), axis=1)
    df['label_yr'] = df.apply(lambda row: label_yr(row), axis=1)
    
    return df

In [7]:
def create_holiday_df():
    
    fes_18 = pd.read_csv('.\data\indian_festivals_2018.csv')
    fes_18['holiday'] = fes_18['holiday'].str.replace(r"\(.*?\)", "", regex=True)
    fes_18.rename(columns={'date': 'bkg_date', 'holiday': 'festival_name'}, inplace=True)
    fes_18['bkg_date'] = pd.to_datetime(fes_18['bkg_date'], format='%Y-%m-%d', errors='coerce')
    fes_18.drop(columns=['day'], inplace=True)

    fes_19 = pd.read_csv('.\data\indian_festivals_2019.csv')
    fes_19['holiday'] = fes_19['holiday'].str.replace(r"\(.*?\)", "", regex=True)
    fes_19.rename(columns={'date': 'bkg_date', 'holiday': 'festival_name'}, inplace=True)
    fes_19['bkg_date'] = pd.to_datetime(fes_19['bkg_date'], format='%Y-%m-%d', errors='coerce')
    fes_19.drop(columns=['day'], inplace=True)

    fes_20 = pd.read_csv('.\data\indian_festivals_2020.csv')
    fes_20['holiday'] = fes_20['holiday'].str.replace(r"\(.*?\)", "", regex=True)
    fes_20.rename(columns={'date': 'bkg_date', 'holiday': 'festival_name'}, inplace=True)
    fes_20['bkg_date'] = pd.to_datetime(fes_20['bkg_date'], format='%Y-%m-%d', errors='coerce')
    fes_20.drop(columns=['day'], inplace=True)

    fes_21 = pd.read_csv('.\data\indian_festivals_2021.csv')
    fes_21['holiday'] = fes_21['holiday'].str.replace(r"\(.*?\)", "", regex=True)
    fes_21.rename(columns={'date': 'bkg_date', 'holiday': 'festival_name'}, inplace=True)
    fes_21['bkg_date'] = pd.to_datetime(fes_21['bkg_date'], format='%Y-%m-%d', errors='coerce')
    fes_21.drop(columns=['day'], inplace=True)

    fes_22 = pd.read_csv('.\data\indian_festivals_2022.csv')
    fes_22[['month_name', 'date']] = fes_22['Date'].str.split(' ', 1, expand=True)
    fes_22.Year = fes_22.Year.astype('str')
    fes_22.month_name = fes_22.month_name.astype('str')
    fes_22.date = fes_22.date.astype('str')
    fes_22['bkg_date'] = pd.to_datetime(fes_22['date'] +'-' +fes_22['month_name']+'-' + fes_22['Year']).dt.strftime('%Y-%m-%d')
    fes_22.drop(columns=['Id', 'Day', 'Date', 'Year', 'month_name', 'date'], inplace=True)
    fes_22.rename(columns={'Festival name': 'festival_name'}, inplace=True)
    fes_22['bkg_date'] = pd.to_datetime(fes_22['bkg_date'], format='%Y-%m-%d', errors='coerce')
    fes_22 = pd.DataFrame(fes_22.groupby('bkg_date')['festival_name'].apply(lambda x: '/'.join(x)).copy())
    fes_22.reset_index(inplace=True)
    fes_22['holiday_type']=0
    key_list = list(fes_21.festival_name)
    dict_lookup = dict(zip(fes_21.festival_name, fes_21.holiday_type))
    fes_22['holiday_type'] = [dict_lookup[item] for item in key_list]

    holiday = pd.concat([fes_18, fes_19, fes_20, fes_21, fes_22])
    holiday = holiday.reset_index(drop=True).copy()
    
    return holiday

holiday = create_holiday_df()

In [8]:
def find_non_rare_labels(df, variable, tolerance):
    
    temp = df.groupby([variable])[variable].count() / len(df)
    
    non_rare = [x for x in temp.loc[temp>tolerance].index.values]
    
    return non_rare



def rare_encoding(X_train, X_test, variable, tolerance):
    
    X_train = X_train.copy()
    X_test = X_test.copy()
    
    # find the mostr frequent category
    frequent_cat = find_non_rare_labels(X_train, variable, tolerance)
    
    # re-group rare labels
    X_train[variable] = np.where(X_train[variable].isin(
           frequent_cat), X_train[variable], 'rare')
    
    X_test[variable] = np.where(X_test[variable].isin(
           frequent_cat), X_test[variable], 'rare')
    
    return X_train, X_test

In [9]:
# count encoding

def categorical_to_counts(df_train, df_valid):
    
    df_train_temp = df_train.copy()
    df_valid_temp = df_valid.copy()
    #df_test_temp = df_test.copy()
    
    for col in ['bkg_date_week', 'bkg_date_month', 'bkg_date_quarter',
                'bkg_date_year',
         'bkg_date_day', 'bkg_date_dayofweek', 'bkg_date_dayname',
                'bkg_date_semester',
         'bkg_date_is_weekend', 'covid_wave', #'festival_name',
                'holiday_type', 'season', 'is_holiday', #'month_name',
                'bkg_date_is_wed_thu',
               'label_mth', 'label_yr', 'week_number', 'holiday_type',
               ]:
        
        # dictionary mapping category to counts
        counts_map = df_train_temp[col].value_counts().to_dict()
        
        # remap the labels to their counts
        df_train_temp[col+'_count'] = df_train_temp[col].map(
            counts_map)
        df_valid_temp[col+'_count'] = df_valid_temp[col].map(counts_map)
#         df_test_temp[col+'_count'] = df_test_temp[col].map(
#             counts_map)
        
    return df_train_temp, df_valid_temp

In [10]:
# ordered integer encoding

def categories_to_ordered(df_train, df_valid, y_train, y_valid):
    
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_valid_temp = pd.concat([df_valid, y_valid], axis=1).copy()
    #df_test_temp = pd.concat([df_test, y_test], axis=1).copy()
    
    for col in ['bkg_date_week', 'bkg_date_month', 'bkg_date_quarter',
                'bkg_date_year',
         'bkg_date_day', 'bkg_date_dayofweek', 'bkg_date_dayname',
                'bkg_date_semester',
         'bkg_date_is_weekend', 'covid_wave', #'festival_name',
                'holiday_type', 'season', 'is_holiday', 'bkg_date_is_wed_thu',
               'label_mth', 'label_yr', 'week_number', 'holiday_type']:
        
        # order categories according to target mean
        ordered_labels = df_train_temp.groupby(
          [col])['weight'].mean().sort_values().index
        
        # dictionary to map ordered labels to ordinal number
        ordinal_label = {k: i for i, k in enumerate(
            ordered_labels, 0)}
        
        # remap the categories to these ordinal numbers
        df_train_temp[col+'_ordinal'] = df_train[col].map(
            ordinal_label)
        df_valid_temp[col+'_ordinal'] = df_valid[col].map(
            ordinal_label)
#         df_test_temp[col+'_ordinal'] = df_test[col].map(
#             ordinal_label)
        
    # remove the target
    df_train_temp.drop(['weight'], axis=1, inplace=True)
    df_valid_temp.drop(['weight'], axis=1, inplace=True)
    #df_test_temp.drop(['count'], axis=1, inplace=True)
    
    return df_train_temp, df_valid_temp

In [11]:
# mean encoding

def categories_to_mean(df_train, df_valid, y_train, y_valid):
    
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_valid_temp = pd.concat([df_valid, y_valid], axis=1).copy()
    #df_test_temp = pd.concat([df_test, y_test], axis=1).copy()
    
    for col in ['bkg_date_week', 'bkg_date_month', 'bkg_date_quarter',
                'bkg_date_year',
         'bkg_date_day', 'bkg_date_dayofweek', 'bkg_date_dayname',
                'bkg_date_semester',
         'bkg_date_is_weekend', 'covid_wave', #'festival_name',
                'holiday_type', 'season', 'is_holiday', 'bkg_date_is_wed_thu',
               'label_mth', 'label_yr', 'week_number', 'holiday_type']:
        
        ordered_labels = df_train_temp.groupby([
            col])['weight'].mean().to_dict()
        
        # remap the categories to target mean
        df_train_temp[col+'_mean_encoding'] = df_train[col].map(
            ordered_labels)
        df_valid_temp[col+'_mean_encoding'] = df_valid[col].map(
            ordered_labels)
#         df_test_temp[col+'_mean_encoding'] = df_test[col].map(
#             ordered_labels)
        
        
    # remove target
    df_train_temp.drop(['weight'], axis=1, inplace=True)
    df_valid_temp.drop(['weight'], axis=1, inplace=True)
   # df_test_temp.drop(['count'], axis=1, inplace=True)
    
    return df_train_temp, df_valid_temp

In [12]:
# remove correlated features to reduce the feature space

def correlation(dataset, threshold):
    # set of all the names of correlated columns
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j])>threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [13]:
span7 = 7
alpha7 = 2/(span7+1)
span14 = 14
alpha14 = 2/(span14+1)
span21 = 21
alpha21 = 2/(span21+1)
span28 = 28
alpha28 = 2/(span28+1)

def mape_score(y_actual, y_pred):
    score = np.round(np.mean(np.abs(y_actual-y_pred)/y_actual)*100,2)
    return score

In [14]:
def lag_feats(df):
    
    temp7 = df.shift(7).copy()
    temp7.rename(columns={'weight': 'weight_lag7'}, 
                     inplace=True)
    temp8 = df.shift(8).copy()
    temp8.rename(columns={'weight': 'weight_lag8'},
                    inplace=True)
    temp9 = df.shift(9).copy()
    temp9.rename(columns={'weight': 'weight_lag9'},
                    inplace=True)
    temp10 = df.shift(10).copy()
    temp10.rename(columns={'weight': 'weight_lag10'},
                     inplace=True)
    temp11 = df.shift(11).copy()
    temp11.rename(columns={'weight': 'weight_lag11'},
                     inplace=True)
    temp12 = df.shift(12).copy()
    temp12.rename(columns={'weight': 'weight_lag12'}, inplace=True)
    temp13 = df.shift(13).copy()
    temp13.rename(columns={'weight': 'weight_lag13'}, inplace=True)
    temp14 = df.shift(14).copy()
    temp14.rename(columns={'weight': 'weight_lag14'}, inplace=True)
    temp21 = df.shift(21).copy()
    temp21.rename(columns={'weight': 'weight_lag21'}, inplace=True)
    temp30 = df.shift(30).copy()
    temp30.rename(columns={'weight': 'weight_lag30'}, inplace=True)
    temp37 = df.shift(37).copy()
    temp37.rename(columns={'weight': 'weight_lag37'}, inplace=True)
    temp44 = df.shift(44).copy()
    temp44.rename(columns={'weight': 'weight_lag44'}, inplace=True)
    temp58 = df.shift(58).copy()
    temp58.rename(columns={'weight': 'weight_lag58'}, inplace=True)
    temp90 = df.shift(90).copy()
    temp90.rename(columns={'weight': 'weight_lag90'}, inplace=True)
    temp180 = df.shift(180).copy()
    temp180.rename(columns={'weight': 'weight_lag180'}, inplace=True)
    temp365 = df.shift(365).copy()
    temp365.rename(columns={'weight': 'weight_lag365'}, inplace=True)
    df1 = pd.concat([df, temp7, temp8, temp9, temp10, temp11, temp12, temp13, temp14, temp21, temp30, temp37, temp44, 
                 temp58, temp90, temp365], axis=1)
    df1.fillna(2, inplace=True)
    df2 = df1.reset_index()
    df = df2.copy()
    
    return df

In [15]:
def weight_cycle_trend(df):
    weight_cycle7, weight_trend7 = hpfilter(df['weight_lag7'], lamb=110930628906.25)
    weight_cycle8, weight_trend8 = hpfilter(df['weight_lag8'], lamb=110930628906.25)
    weight_cycle9, weight_trend9 = hpfilter(df['weight_lag9'], lamb=110930628906.25)
    weight_cycle10, weight_trend10 = hpfilter(df['weight_lag10'], lamb=110930628906.25)
    weight_cycle11, weight_trend11 = hpfilter(df['weight_lag11'], lamb=110930628906.25)
    weight_cycle12, weight_trend12 = hpfilter(df['weight_lag12'], lamb=110930628906.25)
    weight_cycle13, weight_trend13 = hpfilter(df['weight_lag13'], lamb=110930628906.25)
    weight_cycle14, weight_trend14 = hpfilter(df['weight_lag14'], lamb=110930628906.25)
    weight_cycle21, weight_trend21 = hpfilter(df['weight_lag21'], lamb=110930628906.25)
    weight_cycle30, weight_trend30 = hpfilter(df['weight_lag30'], lamb=110930628906.25)
    weight_cycle37, weight_trend37 = hpfilter(df['weight_lag37'], lamb=110930628906.25)
    weight_cycle44, weight_trend44 = hpfilter(df['weight_lag44'], lamb=110930628906.25)
    weight_cycle58, weight_trend58 = hpfilter(df['weight_lag58'], lamb=110930628906.25)
    weight_cycle90, weight_trend90 = hpfilter(df['weight_lag90'], lamb=110930628906.25)
    weight_cycle365, weight_trend365 = hpfilter(df['weight_lag365'], lamb=110930628906.25)
        
    df['weight_cycle7'] = weight_cycle7
    df['weight_trend7'] = weight_trend7
    df['weight_cycle8'] = weight_cycle8
    df['weight_trend8'] = weight_trend8
    df['weight_cycle9'] = weight_cycle9
    df['weight_trend9'] = weight_trend9
    df['weight_cycle10'] = weight_cycle10
    df['weight_trend10'] = weight_trend10
    df['weight_cycle11'] = weight_cycle11
    df['weight_trend11'] = weight_trend11
    df['weight_cycle12'] = weight_cycle12
    df['weight_trend12'] = weight_trend12
    df['weight_cycle13'] = weight_cycle13
    df['weight_trend13'] = weight_trend13
    df['weight_cycle14'] = weight_cycle14
    df['weight_trend14'] = weight_trend14
    df['weight_cycle21'] = weight_cycle21
    df['weight_trend21'] = weight_trend21
    df['weight_cycle30'] = weight_cycle30
    df['weight_trend30'] = weight_trend30
    df['weight_cycle37'] = weight_cycle37
    df['weight_trend37'] = weight_trend37
    df['weight_cycle44'] = weight_cycle44
    df['weight_trend44'] = weight_trend44
    df['weight_cycle58'] = weight_cycle58
    df['weight_trend58'] = weight_trend58
    df['weight_cycle90'] = weight_cycle90
    df['weight_trend90'] = weight_trend90
    df['weight_cycle365'] = weight_cycle365
    df['weight_trend365'] = weight_trend365
    
    return df

In [16]:
def rolling_feats(df):
    
    for col in ['weight_lag13', 'weight_lag8','weight_lag21', 'weight_lag14', 'weight_lag7', 'weight_lag30',
            'weight_lag37', 'weight_lag44',
       'weight_lag58', 'weight_lag365',
       'weight_lag90']:
        
        
        df[col+'_sma7'] = df[col].rolling(window=7).mean()
        df[col+'_sma14'] = df[col].rolling(window=14).mean()
        df[col+'_sma21'] = df[col].rolling(window=21).mean()
        df[col+'_sma28'] = df[col].rolling(window=28).mean()
        df[col+'_ewma7'] = df[col].ewm(span=7).mean()
        df[col+'_ewma14'] = df[col].ewm(span=14).mean()
        df[col+'_ewma21'] = df[col].ewm(span=21).mean()
        df[col+'_ewma28'] = df[col].ewm(span=28).mean()
        df[col+'_rol_std7'] = df[col].rolling(window=7).std()
        df[col+'_rol_std14'] = df[col].rolling(window=14).std()
        df[col+'_rol_std21'] = df[col].rolling(window=21).std()
        df[col+'_rol_std28'] = df[col].rolling(window=28).std()
        df[col+'_rol_ewms7'] = df[col].ewm(span=7).std()
        df[col+'_rol_ewms14'] = df[col].ewm(span=14).std()
        df[col+'_rol_ewms21'] = df[col].ewm(span=21).std()
        df[col+'_rol_ewms28'] = df[col].ewm(span=28).std()

    df.fillna(2, inplace=True)
    df.index.freq = 'D'
        
    for col in ['weight_lag13', 'weight_lag8','weight_lag21', 'weight_lag14', 'weight_lag7', 'weight_lag30',
            'weight_lag37', 'weight_lag44',
       'weight_lag58', 'weight_lag365',
       'weight_lag90']:
        df[col+'_hewma7'] = df[col].ewm(alpha=alpha7, adjust=False).mean()
        df[col+'_hewma14'] = df[col].ewm(alpha=alpha14, adjust=False).mean()
        df[col+'_hewma21'] = df[col].ewm(alpha=alpha21, adjust=False).mean()
        df[col+'_hewma28'] = df[col].ewm(alpha=alpha28, adjust=False).mean()
    
        df[col+'_hewms7'] = df[col].ewm(alpha=alpha7, adjust=False).std()
        df[col+'_hewms14'] = df[col].ewm(alpha=alpha14, adjust=False).std()
        df[col+'_hewms21'] = df[col].ewm(alpha=alpha21, adjust=False).std()
        df[col+'_hewms28'] = df[col].ewm(alpha=alpha28, adjust=False).std()
        
    return df

In [17]:
span7 = 7
alpha7 = 2/(span7+1)
span14 = 14
alpha14 = 2/(span14+1)
span21 = 21
alpha21 = 2/(span21+1)
span28 = 28
alpha28 = 2/(span28+1)

def weight_preds(df):
    train_df = df.iloc[:-7]
    test_df = df.iloc[-7:]
        
    for col in ['weight_lag7',# 'weight_lag8', 'weight_lag9', 'weight_lag10',
       #'weight_lag11', 
                    'weight_lag12', 'weight_lag13', 'weight_lag14',
       'weight_lag21' ,'weight_lag30', 'weight_lag37', 'weight_lag44',
       'weight_lag58', 'weight_lag90', 'weight_lag365']:
        
        model = SimpleExpSmoothing(train_df[col])
        fitted_model = model.fit(smoothing_level=alpha7, optimized=False)
        train_df[col+'_ses7'] = fitted_model.fittedvalues.shift(-1)
        test_df[col+'_ses7'] = fitted_model.forecast(7)
        train_df[col+'_des_add_7'] = ExponentialSmoothing(train_df[col], trend='add').fit().fittedvalues.shift(-1)
        test_df[col+'_des_add_7'] = ExponentialSmoothing(train_df[col], trend='add').fit().forecast(7)
        train_df[col+'_des_mul_7'] = ExponentialSmoothing(train_df[col], trend='mul').fit().fittedvalues.shift(-1)
        test_df[col+'_des_mul_7'] = ExponentialSmoothing(train_df[col], trend='mul').fit().forecast(7)
        train_df[col+'_tes_mul_7'] = ExponentialSmoothing(train_df[col], trend='mul', seasonal='mul', 
                                             seasonal_periods=7).fit().fittedvalues
        test_df[col+'_tes_mul_7'] = ExponentialSmoothing(train_df[col], trend='mul', seasonal='mul',
                                            seasonal_periods=7).fit().forecast(7)

    train_df.fillna(2, inplace=True)
        
    start = len(train_df)
    end = len(train_df) + len(test_df) - 1
        
    for col in ['weight_lag7', 'weight_lag8', 'weight_lag9', 'weight_lag10',
       'weight_lag11', 'weight_lag12', 'weight_lag13', 'weight_lag14',
       'weight_lag21' ,'weight_lag30', 'weight_lag37', 'weight_lag44',
       'weight_lag58', 'weight_lag90', 'weight_lag365']:
        
        
        AR1fit = AutoReg(train_df[col], lags=1).fit()
        train_df[col+'_ar1_preds'] = AR1fit.fittedvalues
        test_df[col+'_ar1_preds'] =  AR1fit.predict(start=start, end=end)
        AR7fit = AutoReg(train_df[col], lags=7).fit()
        train_df[col+'_ar7_preds'] = AR7fit.fittedvalues
        test_df[col+'_ar7_preds'] =  AR7fit.predict(start=start, end=end)
        AR30fit = AutoReg(train_df[col], lags=30).fit()
        train_df[col+'_ar30_preds'] = AR30fit.fittedvalues
        test_df[col+'_ar30_preds'] =  AR30fit.predict(start=start, end=end)
        AR365fit = AutoReg(train_df[col], lags=365).fit()
        train_df[col+'_ar365_preds'] = AR365fit.fittedvalues
        test_df[col+'_ar365_preds'] =  AR365fit.predict(start=start, end=end)

    for col in ['weight_lag7', 'weight_lag8', 'weight_lag9', 'weight_lag10',
       'weight_lag11', 'weight_lag12', 'weight_lag13', 'weight_lag14',
       'weight_lag21' ,'weight_lag30', 'weight_lag37', 'weight_lag44',
       'weight_lag58', 'weight_lag90', 'weight_lag365']:
        
        
        model = sm.tsa.ARIMA(train_df[col], order=(4,1,5))
        model.initialize_approximate_diffuse()
        model = model.fit()
        train_df[col+'_arima_preds'] = model.fittedvalues
        test_df[col+'_arima_preds'] = model.predict(start=start, end=end)
        model = sm.tsa.ARIMA(train_df[col], order=(0,0,1))
        model.initialize_approximate_diffuse()
        model = model.fit()
        train_df[col+'_ma_preds'] = model.fittedvalues
        test_df[col+'_ma_preds'] = model.predict(start=start, end=end)
        model = sm.tsa.ARIMA(train_df[col], order=(2,0,1))
        model.initialize_approximate_diffuse()
        model = model.fit()
        train_df[col+'_arma_preds'] = model.fittedvalues
        test_df[col+'_arma_preds'] = model.predict(start=start, end=end)

    return train_df, test_df

In [ ]:
# def sarimax_preds(df):
    
#     # drop categorical columns
#     df1 = df.drop(columns=[])
#     train_df = df.iloc[:-7]
#     test_df = df.iloc[-7:]

#     model = SARIMAX(train_df['weight'], exog=train_df[X_train.columns],
#                    order=(1, 1, 1), seasonal_order=(1, 0, [1, 2], 7),
#                    enforce_invertibility=False)
    
#     result = model.fit()
#     start = 0
#     end = len(train_df) + len(test_df) - 1
#     sarimax_preds = result.predict(start, end, )
    


# # model = SARIMAX(train_df['weight'], exog=train_df[X_train.columns], 
# #                 order=(1, 1, 1), seasonal_order=(1, 0, [1,2], 7),
# #                enforce_invertibility=False)
# # result = model.fit()
# # start = len(train_df)
# # end = len(train_df) + len(test_df) - 1
# # test_predictions1 = result.predict(start, end, exog=test_df[X_train.columns]).rename('SARIMAX with exog1')
# # start = 0
# # end = len(train_df)-1
# # train_predictions1 = result.predict(start, end, exog=train_df[X_train.columns]).rename('SARIMAX with exog1')
# # X_train['sarimax_preds'] = train_predictions1
# # X_valid['sarimax_preds'] = test_predictions1

In [18]:
top_80 = ['DELH_LKOD','BLRO_LKOD','DELH_MAAO','DELH_CCUO','AMDO_SLMO','DELH_NAGO','AMC_AMDO','AMC_SLMO','DELH_PNQO','DELH_IDRO',
 'DELH_BLRO','AMDO_BLRO','CCUO_DELH','AMC_CCUO','DELH_PATO','DELH_HYDO','BLRO_SLMO','BWDV_DELH','PNQO_AMDO','DELH_NDAO',
 'CCUO_SLMO','BLRO_MAAO','CCUO_BBIO','DELH_JAIO','DELH_AMC','DELH_GUAO','AMC_NDAO','AMC_PNQO','AMC_DELH','BLRO_PNQO',
 'PNQO_CCUO','AMDO_AMC','BLRO_NDAO','MAAO_HYDO','AMC_JAIO','DELH_BWDV','NDAO_AMC','BLRO_AMC','CCUO_PATO','AMC_BWDV',
 'NDAO_SLMO','AMDO_DELH','PNQO_BLRO','AMDO_CCUO','AMC_LKOD','AMDO_PNQO','DELH_AMDO','AMC_IDRO','AMC_BLRO','AMC_MAAO',
 'BWDV_BLRO','BLRO_DELH','PNQO_DELH','CCUO_PNQO','AMC_HYDO','PNQO_MAAO','AMC_PATO','AMC_NAGO','BLRO_JAIO','MAAO_SLMO',
 'DELH_SLMO','BLRO_VZAO','NDAO_CCUO','BLRO_AMDO','MAAO_VZAO','DELH_BBIO','CCUO_BLRO','BLRO_HYDO','NDAO_LKOD',
 'BLRO_CCUO','MAAO_AMDO','BWDV_PNQO','BWDV_AMDO','PNQO_AMC','PNQO_SLMO','SLMO_DELH','BWDV_CCUO','BWDV_AMC','MAAO_BLRO',
 'BWDV_SLMO','CCUO_GUAO','MAAO_DELH']

In [25]:
def each_stc(df1, tail_length):
    count = 1
    results = pd.DataFrame()
    for idx, v in enumerate(top_80):
        df = df1.loc[df1['stc_lane']==v].copy()
        df = df[:-30].copy()
        
        df = df.iloc[:(len(df) - tail_length + 7)]
        stc = df[['stc_lane']].copy()
        bkg_date = df[['bkg_date']].copy()
        df.drop(columns=['stc_lane'], inplace=True)
        
        if((len(df[df['weight']==0])/(len(df))*100)>50):
            print(v, 'more than 50% weight is 0')
            df['weight'] = np.where(df['weight']==0, np.nan, df.weight)
            df = df.fillna(method='ffill').fillna(method='bfill')
            df.reset_index(drop=True, inplace=True)
            df.set_index('bkg_date', inplace=True)
            df.sort_index(inplace=True)
        else:
            dfy = df[df['bkg_date']<'2019-12-31'].copy()
            dfy.reset_index(drop=True, inplace=True)
            for col in ['bkg_date']:
                dfy[col+'_week'] = dfy[col].dt.week
                dfy[col+'_month'] = dfy[col].dt.month
                dfy[col+'_year'] = dfy[col].dt.year
                dfy[col+'_day'] = dfy[col].dt.day
                dfy[col+'_dayofweek'] = dfy[col].dt.dayofweek
                dfy[col+'_day_of_year'] = dfy[col].dt.dayofyear
                dfy[col+'_week_of_year'] = dfy[col].dt.weekofyear
                
            dfy['weight'] = np.where(dfy['weight']==0, np.nan, dfy.weight)
            dfy2 = imputer.fit_transform(dfy[['weight', 'bkg_date_week', 'bkg_date_month',
                  'bkg_date_year', 'bkg_date_day', 'bkg_date_dayofweek',
                  'bkg_date_day_of_year', 'bkg_date_week_of_year']])
            dfy3 = pd.DataFrame(data=dfy2, columns=['weight', 'bkg_date_week', 'bkg_date_month',
                   'bkg_date_year', 'bkg_date_day', 'bkg_date_dayofweek',
                    'bkg_date_day_of_year', 'bkg_date_week_of_year'])
            dfy4 = pd.concat([dfy['bkg_date'], dfy3], axis=1)
            dfy4.set_index('bkg_date', inplace=True)
            dfy4.sort_index(inplace=True)
            dfa = df[df['bkg_date']<'2019-12-31'][df['weight']==0]
            dfa.set_index('bkg_date', inplace=True)
            dfa.sort_index(inplace=True)
            a = set(dfa.index)
            dfa.loc[a, 'weight'] = dfy4['weight']
            dfa.sort_index(inplace=True)
            
            dfz = df[df['bkg_date']>='2019-12-31'][df['weight']==0].copy()
            dfz.reset_index(drop=True, inplace=True)
            dfz.bkg_date = dfz.bkg_date - pd.DateOffset(years=1)
            key_list = list(dfz.bkg_date)
            dict_lookup = dict(zip(df.bkg_date, df.weight))
            dfz['weight'] = [dict_lookup[item] for item in key_list]
            dfz['weight'] = dfz['weight']*0.76
            dfz.bkg_date = dfz.bkg_date + pd.DateOffset(years=1)
            df.set_index('bkg_date', inplace=True)
            df.sort_index(inplace=True)
            dfz.set_index('bkg_date', inplace=True)
            dfz.sort_index(inplace=True)
            a = set(dfz.index)
            df.loc[a, 'weight'] = dfz['weight']
            b = set(dfa.index)
            df.loc[b, 'weight'] = dfa['weight']
            
        df.sort_index(inplace=True)
        df['weight'] = np.ceil(df['weight']+1)
        df = lag_feats(df)
        df = datetime_feats(df)
        
        stocks = pd.read_pickle('stocks7_oct18_nov22.pkl')
        stocks['bkg_date'] = stocks.index
        stocks['bkg_date'] = pd.to_datetime(stocks['bkg_date'], 
                                    format='%Y-%m-%d', errors='coerce')
        stocks.reset_index(drop=True, inplace=True)
        df = pd.merge(df, stocks, how = 'left', on='bkg_date').copy()

        df = df.merge(holiday, how='left', on='bkg_date').copy()
        df.fillna('working_day', inplace=True)
        df.drop(columns=['festival_name'], inplace=True)
        
        df['is_holiday'] = np.where((df['holiday_type']!='working_day'), 1, 0)
        df['is_holiday_restricted'] = np.where((df['holiday_type']=='Restricted'), 1, 0)
        df['is_holiday_gazetted'] = np.where((df['holiday_type']=='Gazetted'), 1, 0)
        #bkg_date = df[['bkg_date']].copy()
        df.set_index('bkg_date', inplace=True)
        
        df.sort_index(inplace=True)
        df = weight_cycle_trend(df)
        df = rolling_feats(df)
        df.fillna(2, inplace=True)
        train_df, test_df = weight_preds(df)
        train_df.fillna(2, inplace=True)
        test_df.fillna(2, inplace=True)
        
        X_train = train_df.drop(labels=['weight'], axis=1)
        y_train = train_df['weight']
        X_valid = test_df.drop(labels=['weight'], axis=1)
        y_valid = test_df['weight']
        
        X_train_count, X_valid_count = categorical_to_counts(X_train, X_valid)
        X_train_ordered, X_valid_ordered = categories_to_ordered(X_train_count, X_valid_count, y_train, y_valid)
        X_train_mean, X_valid_mean = categories_to_mean(X_train_ordered, X_valid_ordered, y_train, y_valid)
        
        for col in ['bkg_date_week', 'bkg_date_month', 'bkg_date_quarter', 'bkg_date_year',
                         'bkg_date_day', 'bkg_date_dayofweek', 'bkg_date_semester', 'bkg_date_is_weekend',
                          'weight_lag30', 'weight_lag37', 'weight_lag44', 'weight_lag58',
                           'weight_lag365', 'weight_lag90']:
            X_train_mean[col] = X_train_mean[col].astype('float')
            X_valid_mean[col] = X_valid_mean[col].astype('float')
            X_train_mean[col+'_yeo'], param = stats.yeojohnson(X_train_mean[
                col])
            X_valid_mean[col+'_yeo']= stats.yeojohnson(X_valid_mean[col], param)
            
        X_train_mean.drop(columns=['bkg_date_dayname', 
                               'covid_wave', 
                               'holiday_type', 'label_mth', 'label_yr'
                              ], inplace=True)
        X_valid_mean.drop(columns=['bkg_date_dayname', 
                                   'covid_wave', 
                               'holiday_type', 'label_mth', 'label_yr'
                              ], inplace=True)
        
        X_train_mean = pd.DataFrame(X_train_mean)
        X_valid_mean = pd.DataFrame(X_valid_mean)
        X_train = X_train_mean.copy()
        X_valid = X_valid_mean.copy()
        train_data = pd.concat([X_train, y_train], axis=1)
        valid_data = pd.concat([X_valid, y_valid], axis=1)
        final = pd.concat([train_data, valid_data])
        print('shape of data without stc', final.shape)
        
        stc.index = final.index
        bkg_date.index = final.index
        final_1 = pd.concat([final, stc], axis=1)
        final_1 = pd.concat([final_1, bkg_date], axis=1)
        print('shape of data after adding stc and bkg_date', final_1.shape)
        final = pd.DataFrame(final_1)
        results = pd.concat([results, final], axis=0).reset_index(drop=True)
        print('shape of returned data', results.shape)
        count = count+1
        print('Running on ', count, 'lane out of ', len(top_80))
        
            
    return results

In [ ]:
%%time
res7 = each_stc(df, 7)
res14 = each_stc(df, 14)
res21 = each_stc(df, 21)
res28 = each_stc(df, 28)
res35 = each_stc(df, 35)

shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (1431, 551)
Running on  2 lane out of  82
shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (2862, 551)
Running on  3 lane out of  82
shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (4293, 551)
Running on  4 lane out of  82
shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (5724, 551)
Running on  5 lane out of  82
shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (7155, 551)
Running on  6 lane out of  82
shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (8586, 551)
Running on  7 lane out of  82
shape of data without stc (1431, 549)
shape of

shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (74412, 551)
Running on  53 lane out of  82
shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (75843, 551)
Running on  54 lane out of  82
shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (77274, 551)
Running on  55 lane out of  82
shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (78705, 551)
Running on  56 lane out of  82
shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (80136, 551)
Running on  57 lane out of  82
shape of data without stc (1431, 549)
shape of data after adding stc and bkg_date (1431, 551)
shape of returned data (81567, 551)
Running on  58 lane out of  82
shape of data without stc (1431, 5

shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (29904, 551)
Running on  22 lane out of  82
shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (31328, 551)
Running on  23 lane out of  82
shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (32752, 551)
Running on  24 lane out of  82
shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (34176, 551)
Running on  25 lane out of  82
shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (35600, 551)
Running on  26 lane out of  82
shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (37024, 551)
Running on  27 lane out of  82
shape of data without stc (1424, 5

shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (102528, 551)
Running on  73 lane out of  82
shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (103952, 551)
Running on  74 lane out of  82
shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (105376, 551)
Running on  75 lane out of  82
shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (106800, 551)
Running on  76 lane out of  82
shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (108224, 551)
Running on  77 lane out of  82
shape of data without stc (1424, 549)
shape of data after adding stc and bkg_date (1424, 551)
shape of returned data (109648, 551)
Running on  78 lane out of  82
shape of data without stc (1

shape of data without stc (1417, 549)
shape of data after adding stc and bkg_date (1417, 551)
shape of returned data (58097, 551)
Running on  42 lane out of  82
shape of data without stc (1417, 549)
shape of data after adding stc and bkg_date (1417, 551)
shape of returned data (59514, 551)
Running on  43 lane out of  82
shape of data without stc (1417, 549)
shape of data after adding stc and bkg_date (1417, 551)
shape of returned data (60931, 551)
Running on  44 lane out of  82
shape of data without stc (1417, 549)
shape of data after adding stc and bkg_date (1417, 551)
shape of returned data (62348, 551)
Running on  45 lane out of  82
shape of data without stc (1417, 549)
shape of data after adding stc and bkg_date (1417, 551)
shape of returned data (63765, 551)
Running on  46 lane out of  82
shape of data without stc (1417, 549)
shape of data after adding stc and bkg_date (1417, 551)
shape of returned data (65182, 551)
Running on  47 lane out of  82
shape of data without stc (1417, 5

shape of data without stc (1410, 549)
shape of data after adding stc and bkg_date (1410, 551)
shape of returned data (14100, 551)
Running on  11 lane out of  82
shape of data without stc (1410, 549)
shape of data after adding stc and bkg_date (1410, 551)
shape of returned data (15510, 551)
Running on  12 lane out of  82
shape of data without stc (1410, 549)
shape of data after adding stc and bkg_date (1410, 551)
shape of returned data (16920, 551)
Running on  13 lane out of  82
shape of data without stc (1410, 549)
shape of data after adding stc and bkg_date (1410, 551)
shape of returned data (18330, 551)
Running on  14 lane out of  82
shape of data without stc (1410, 549)
shape of data after adding stc and bkg_date (1410, 551)
shape of returned data (19740, 551)
Running on  15 lane out of  82
shape of data without stc (1410, 549)
shape of data after adding stc and bkg_date (1410, 551)
shape of returned data (21150, 551)
Running on  16 lane out of  82
shape of data without stc (1410, 5

* res7 - test data is last week of august
* re14 - test data is 2nd last week of august
* re21 - test data is 2nd week of august
* res28 - test data is 1st week of august
* res35 - test data is last week of july

In [22]:
res7.shape, res14.shape, res21.shape, res28.shape

((2862, 551), (2848, 551), (2834, 551), (2820, 551))

In [23]:
# res.to_pickle('master_data_top_80.pkl')

In [24]:
df.shape

(644301, 3)